<b> Importing the necessary libraries</b>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import os
import json
import sys
from pprint import pprint
import pandas as pd
from datetime import timedelta
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

<b>Setting current working directory absolute path</b>

In [2]:
temp = os.getcwd().split('/')
path = '/'.join(temp[:(len(temp))])

<b>Reading and initializing the config values</b>

In [3]:
try:
    with open(path+'/token.json') as file:
        config = json.load(file)
except Exception:
    print('Config file not found / JSON format wrong in file')
    sys.exit(0)

<b>Defining the <i>token</i>, <i>params</i> and <i>headers</i> for the requests</b>

In [4]:
token = os.getenv('GITHUB_TOKEN', config['token'])
parameters = {"state" : "all",}
headers = {'Authorization' : f'token {token}'}

<b>Passing the name of repositories whose issues are to be visualised in a list</b>

In [5]:
repo = ['mvp-icap-service', 'mvp-icap-cloud', 'mvp-icap-squid-cache-proxy', 'rebuild-k8s-filetypedetection',\
       'icap-performance-tests', 'rebuild-k8s']

<b>Fetching the issues data in JSON from the respective reposetories and appending them to a list</b>

In [ ]:
data = []
for i in range(0, len(repo)):
    data.append(requests.get('https://api.github.com/repos/filetrust/'+repo[i]+'/issues', headers = headers, params = parameters).json())

<b>Normalizing the JSON objects in order to convert them to DataFrames and appending them to another list</b>

In [ ]:
dataframes = []
for i in range(0, len(data)):
    dataframes.append(json_normalize(data[i], max_level = 1))

<b>Adding <i>project_name</i> attribite and concatenating the DataFrames in order to get a single DataFrame of all the rows</b>

In [ ]:
for i in range(0, len(dataframes)):
    dataframes[i]['project_name'] = repo[i]
df = pd.concat(dataframes, ignore_index = True)

<b>Getting rid of irrelevent columns</b>

In [ ]:
df= df[['created_at', 'state', 'closed_at','user.login','author_association', 'title','body', 'milestone.title','milestone.state','project_name']]

<b>Extracting the creation and closure dates from the datetime variable <i>created_at</i> and <i>closed_at</i></b>

In [ ]:
df.created_at = pd.to_datetime(df.created_at).dt.date
df.closed_at = pd.to_datetime(df.closed_at).dt.date

<b>Creating a variable <i>project_duration</i> to reflect the total duration of the project in days<b>

In [ ]:
df['project_duration'] = (df.closed_at - df.created_at) + timedelta(days=1)

In [ ]:
df.sample(5)

<b> Testing the module</b>

In [ ]:
import IssuesData
repos = ['mvp-icap-service', 'mvp-icap-squid-cache-proxy', 'rebuild-k8s-filetypedetection',\
       'icap-performance-tests', 'rebuild-k8s']
df = IssuesData.get_issues(repo = repos)
df.shape